# 식신-제주시청

In [66]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup
import parser

### 1. 데이터 가져오기

In [84]:
site = 'https://www.siksinhot.com'
url = f'{site}/search?keywords={quote("제주시청")}'
req = requests.get(url)
html = req.text
#html


In [85]:
soup = BeautifulSoup(html,'html.parser')

### 2. 찾으려고 하는 데이터의 태그 찾기

In [86]:
lis = soup.select('div.listTy1>ul>li')
len(lis)

12

### 3. 하나를 선택해서 원하는 정보 추출
- 상호 평점 메뉴 서브페이지의 url
- 전화번호 주소

In [91]:
li = lis[1]
href = li.find('a')['href']
href

'/P/466268'

In [88]:
score = li.select_one('.score').string
score

'3.4'

In [89]:
store = li.select_one('.store').get_text()
store

'안양돈까스 제주시점'

In [90]:
menu = li.find('p').get_text()
menu

'돈까스 세트, 치즈돈까스 세트'

In [92]:
sub_url = site+href
sub_url

'https://www.siksinhot.com/P/466268'

In [94]:
sub_req = requests.get(sub_url)
sub_html = sub_req.text
sub_soup = BeautifulSoup(sub_html,'html.parser')

In [96]:
tel = sub_soup.select_one('.p_tel').find('p').get_text()
tel

'064-799-1434'

In [98]:
addr = sub_soup.select_one('.txt_adr').get_text()
addr

'제주특별자치도 제주시 애월읍 애월로 40'

### 4.페이지에 있는 데이터 가져오기

In [102]:
#try - except 을 이용해서 어디에서 에러가 발생했는지 확인

lines = []
for index, li in enumerate(lis):
    try:
        href = li.find('a')['href']
        score = li.select_one('.score').string
        store = li.select_one('.store').get_text()
        menu = li.find('p').get_text()


        sub_url = site+href
        sub_req = requests.get(sub_url)
        sub_html = sub_req.text
        sub_soup = BeautifulSoup(sub_html,'html.parser')
        tel = sub_soup.select_one('.p_tel').find('p').get_text()
        addr = sub_soup.select_one('.txt_adr').get_text()
    except:
        print(index)

    lines.append({store,score,menu,tel,addr})

0


### 5. 에러난 부분을 처리할 수 있도록 수정

In [112]:
li = lis[0]
score = li.select_one('.cnt > em').get_text()
score

'평가중'

In [113]:
lis[1].select_one('.cnt > em').get_text()

'3.4'

In [132]:
lines = []
for index, li in enumerate(lis):
    try:
        href = li.find('a')['href']
        score = li.select_one('.cnt > em').get_text()
        if score =='평가중':
            score = '평가중'
        store = li.select_one('.store').get_text()
        menu = li.find('p').get_text()


        sub_url = site+href
        sub_req = requests.get(sub_url)
        sub_html = sub_req.text
        sub_soup = BeautifulSoup(sub_html,'html.parser')
        tel = sub_soup.select_one('.p_tel').find('p').get_text()
        addr = sub_soup.select_one('.txt_adr').get_text()
    except:
        print(index)

    lines.append([store,score,menu,tel,addr])

### 데이터 프레임으로 만들기

In [133]:
df = pd.DataFrame(lines, columns=['업소명', '평점', '메뉴', '전화번호', '주소'])

In [134]:
df

,업소명,평점,메뉴,전화번호,주소
0,제주시새우리,평가중,"딱새우김밥, 간장새우김밥, 양념새우컵밥, 딱새우꼬막무침, 한라봉에이드",064-900-2527,제주특별자치도 제주시 무근성7길 21
1,안양돈까스 제주시점,3.4,"돈까스 세트, 치즈돈까스 세트",064-799-1434,제주특별자치도 제주시 애월읍 애월로 40
2,올래국수,3.8,고기국수,064-742-7355,제주특별자치도 제주시 귀아랑길 24
3,명진전복,3.8,"전복죽, 전복돌솥밥, 전복구이, 전복회",064-782-9944,제주특별자치도 제주시 구좌읍 해맞이해안로 1282
4,삼성혈해물탕,3.9,"해물탕(특대), 해물탕(대), 해물탕(중), 해물탕(소), 전복회(소)",064-745-3000,제주특별자치도 제주시 선덕로5길 20
5,자매국수,3.9,"멸치국수, 고기국수, 비빔국수, 멸/고기국수, 아강발(미니족박), 콩국수",064-746-2222,제주특별자치도 제주시 탑동로11길 6
6,해녀촌,3.7,"회국수, 성게국수, 소라국수",064-783-5438,제주특별자치도 제주시 구좌읍 동복로 33
7,순옥이네명가,3.8,"소라, 성게, 한치물회, 순옥이네물회, 전복물회",064-743-4813,제주특별자치도 제주시 도공로 8
8,은희네해장국,3.9,"소고기해장국, 해장국, 내장탕",064-726-5622,
9,삼대국수회관 본점,3.9,"고기국수, 비빔국수, 돔베고기 세트, 물만두 세트",064-759-6645,제주특별자치도 제주시 삼성로 41
